In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

# **Gemma_2b**

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_new_tokens = 600
# max_seq_length = 1024
# dtype = None
# load_in_4bit = True

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/gemma-2b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

# **Gemma_7b**

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_new_tokens = 600
# max_seq_length = 1024
# dtype = None
# load_in_4bit = True

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/gemma-7b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#)

# **Gemma 7b Finetuned**

In [ ]:
# from unsloth import FastLanguageModel
# from datasets import load_dataset

# max_new_tokens = 600
# max_seq_length = 1024
# dtype = None
# load_in_4bit = True

# data_col = "question"

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "/content/drive/MyDrive/NLP/gemma_spider_lora_model_2",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )


# **Gemma 2b Finetuned**

In [ ]:
#Megha
from unsloth import FastLanguageModel
from datasets import load_dataset

max_new_tokens = 600
max_seq_length = 1024
dtype = None
load_in_4bit = True

data_col = "question"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/NLP/gemma2B_spider_lora_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Unsloth 2024.5 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [ ]:
import pandas as pd
spider_prompt = """"Write a query that answers the following question. Return only the query and no additional explanation."

<question> {} <query> {}"""

def formatting_prompts_func(examples):
    questions = examples["question"]
    texts = []
    for question in questions:
        text = spider_prompt.format(question, "")
        texts.append(text)
    return { "modified question" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("xlangai/spider", split = "validation")
test_df = pd.DataFrame(dataset)
display(test_df.head())
test_df.info()


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]"
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]"
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                1034 non-null   object
 1   query                1034 non-null   object
 2   question             1034 non-null   object
 3   query_toks           1034 non-null   object
 4   query_toks_no_value  1034 non-null   object
 5   question_toks        1034 non-null   object
dtypes: object(6)
memory usage: 48.6+ KB


In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)
mod_test_df = pd.DataFrame(dataset)
display(mod_test_df.head())

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]","""Write a query that answers the following ques..."
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]","""Write a query that answers the following ques..."
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin...","""Write a query that answers the following ques..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ...","""Write a query that answers the following ques..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m...","""Write a query that answers the following ques..."


In [ ]:
mod_test_df.shape

(1034, 7)

In [ ]:
import numpy as np
num_splits = 20
mod_test_df_split = np.array_split(mod_test_df, num_splits)


In [ ]:
max_new_tokens = 600
max_seq_length = 1024

output = []
for df in mod_test_df_split:
  FastLanguageModel.for_inference(model)
  print("="*100)
  print(df.shape)
  questions = [i.replace("<eos>","") for i in list(df["modified question"])]
  original_queries = list(df["query"])
  print("Input: ",questions)

  # Tokenize all questions at once with truncation
  tokenizer.padding_side = "left"
  inputs = tokenizer(questions, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt").to("cuda")
  # inputs = tokenizer(questions, return_tensors="pt").to("cuda")
  # inputs = tokenizer(inputs, return_tensors = "pt", padding = True).to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = max_new_tokens, use_cache = True)

  generated_queries = []
  decoded = tokenizer.batch_decode(outputs)
  for i,text in enumerate(decoded):
    print(f"Original query:\n{original_queries[i]}")
    generated_query = text.replace(tokenizer.pad_token, "").split("<query>")[-1]
    generated_queries.append(generated_query.replace("<eos>",""))
    print(f"Generated query:\n{generated_query}")
    print("_" * 100)
  df["generated_query"] = generated_queries
  display(df)
  output.append(df)

(52, 7)
Input:  ['"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> How many singers do we have? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the total number of singers? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Show name, country, age for all singers ordered by age from the oldest to the youngest. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What are the names, countries, and ages for every singer in descending order of age? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the average, minimum, and maximum age of all singers from France? <query> ', '"Write a 

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question,generated_query
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]","""Write a query that answers the following ques...",\n\nSELECT COUNT(*) FROM singers
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]","""Write a query that answers the following ques...",\n\nSELECT SUM(singer_count) FROM singers
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin...","""Write a query that answers the following ques...","<strong>SELECT name, country, age FROM singer..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ...","""Write a query that answers the following ques...","\n\nSELECT singer_name , country , age FR..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m...","""Write a query that answers the following ques...","\n\nSELECT AVG(age) , MIN(age) , MAX(age) FRO..."
5,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m...","""Write a query that answers the following ques...","<strong>SELECT AVG(age), MIN(age), MAX(age) F..."
6,concert_singer,"SELECT song_name , song_release_year FROM sin...",Show the name and the release year of the song...,"[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...","[Show, the, name, and, the, release, year, of,...","""Write a query that answers the following ques...","<strong>SELECT</strong> name , release_year..."
7,concert_singer,"SELECT song_name , song_release_year FROM sin...",What are the names and release years for all t...,"[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...","[What, are, the, names, and, release, years, f...","""Write a query that answers the following ques...","\n\nSELECT name , release_year FROM songs ..."
8,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,What are all distinct countries where singers ...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...","[What, are, all, distinct, countries, where, s...","""Write a query that answers the following ques...",\n\nSELECT DISTINCT country FROM singers WHER...
9,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,What are the different countries with singers...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...","[What, are, the, different, countries, with, s...","""Write a query that answers the following ques...",\n\nSELECT DISTINCT country FROM singers WHER...


(52, 7)
Input:  ['"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> How many pets are owned by students that have an age greater than 20? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Find the number of dog pets that are raised by female students (with sex F). <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> How many dog pets are raised by female students? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Find the number of distinct type of pets. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> How many different types of pet are there? <query> ', '"Write a query that answers the following

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question,generated_query
52,pets_1,SELECT count(*) FROM student AS T1 JOIN has_pe...,How many pets are owned by students that have ...,"[SELECT, count, (, *, ), FROM, student, AS, T1...","[select, count, (, *, ), from, student, as, t1...","[How, many, pets, are, owned, by, students, th...","""Write a query that answers the following ques...",\n\nSELECT count(*) FROM pets WHERE student_i...
53,pets_1,SELECT count(*) FROM student AS T1 JOIN has_pe...,Find the number of dog pets that are raised by...,"[SELECT, count, (, *, ), FROM, student, AS, T1...","[select, count, (, *, ), from, student, as, t1...","[Find, the, number, of, dog, pets, that, are, ...","""Write a query that answers the following ques...",<strong>SELECT count(*) FROM pet WHERE sex = ...
54,pets_1,SELECT count(*) FROM student AS T1 JOIN has_pe...,How many dog pets are raised by female students?,"[SELECT, count, (, *, ), FROM, student, AS, T1...","[select, count, (, *, ), from, student, as, t1...","[How, many, dog, pets, are, raised, by, female...","""Write a query that answers the following ques...",\n\nSELECT count(*) FROM pet WHERE gender = '...
55,pets_1,SELECT count(DISTINCT pettype) FROM pets,Find the number of distinct type of pets.,"[SELECT, count, (, DISTINCT, pettype, ), FROM,...","[select, count, (, distinct, pettype, ), from,...","[Find, the, number, of, distinct, type, of, pe...","""Write a query that answers the following ques...",\n\nSELECT DISTINCT type FROM pets
56,pets_1,SELECT count(DISTINCT pettype) FROM pets,How many different types of pet are there?,"[SELECT, count, (, DISTINCT, pettype, ), FROM,...","[select, count, (, distinct, pettype, ), from,...","[How, many, different, types, of, pet, are, th...","""Write a query that answers the following ques...",\n\nSELECT DISTINCT pet_type FROM pets
57,pets_1,SELECT DISTINCT T1.Fname FROM student AS T1 JO...,Find the first name of students who have cat o...,"[SELECT, DISTINCT, T1.Fname, FROM, student, AS...","[select, distinct, t1, ., fname, from, student...","[Find, the, first, name, of, students, who, ha...","""Write a query that answers the following ques...",<strong>SELECT first_name FROM student WHERE ...
58,pets_1,SELECT DISTINCT T1.Fname FROM student AS T1 JO...,What are the first names of every student who ...,"[SELECT, DISTINCT, T1.Fname, FROM, student, AS...","[select, distinct, t1, ., fname, from, student...","[What, are, the, first, names, of, every, stud...","""Write a query that answers the following ques...",\n\nSELECT first_name FROM student WHERE pet_...
59,pets_1,select t1.fname from student as t1 join has_pe...,Find the first name of students who have both ...,"[select, t1.fname, from, student, as, t1, join...","[select, t1, ., fname, from, student, as, t1, ...","[Find, the, first, name, of, students, who, ha...","""Write a query that answers the following ques...",<strong>SELECT first_name FROM student WHERE ...
60,pets_1,SELECT T1.Fname FROM student AS T1 JOIN has_pe...,What are the students' first names who have bo...,"[SELECT, T1.Fname, FROM, student, AS, T1, JOIN...","[select, t1, ., fname, from, student, as, t1, ...","[What, are, the, students, ', first, names, wh...","""Write a query that answers the following ques...",\n\nSELECT first_name FROM students WHERE pet...
61,pets_1,"SELECT major , age FROM student WHERE stuid N...",Find the major and age of students who do not ...,"[SELECT, major, ,, age, FROM, student, WHERE, ...","[select, major, ,, age, from, student, where, ...","[Find, the, major, and, age, of, students, who...","""Write a query that answers the following ques...","<strong>SELECT major , age FROM student WHERE..."


(52, 7)
Input:  ['"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What are the different models for the cards produced after 1980? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> How many car makers are there in each continents? List the continent name and the count. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the name of each continent and how many car makers are there in each one? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Which of the countries has the most car makers? List the country name. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the name of the cou

...

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question,generated_query
881,network_1,SELECT grade FROM Highschooler GROUP BY grade ...,Which grades have 4 or more high schoolers?,"[SELECT, grade, FROM, Highschooler, GROUP, BY,...","[select, grade, from, highschooler, group, by,...","[Which, grades, have, 4, or, more, high, schoo...","""Write a query that answers the following ques...",\n\nSELECT * FROM student_grade WHERE grad...
882,network_1,"SELECT student_id , count(*) FROM Friend GROU...",Show the student IDs and numbers of friends co...,"[SELECT, student_id, ,, count, (, *, ), FROM, ...","[select, student_id, ,, count, (, *, ), from, ...","[Show, the, student, IDs, and, numbers, of, fr...","""Write a query that answers the following ques...","\n\nSELECT student_id , count(*) AS number_..."
883,network_1,"SELECT student_id , count(*) FROM Friend GROU...",How many friends does each student have?,"[SELECT, student_id, ,, count, (, *, ), FROM, ...","[select, student_id, ,, count, (, *, ), from, ...","[How, many, friends, does, each, student, have...","""Write a query that answers the following ques...",\n\nSELECT count(*) FROM student
884,network_1,"SELECT T2.name , count(*) FROM Friend AS T1 J...",Show the names of high school students and the...,"[SELECT, T2.name, ,, count, (, *, ), FROM, Fri...","[select, t2, ., name, ,, count, (, *, ), from,...","[Show, the, names, of, high, school, students,...","""Write a query that answers the following ques...","<strong>SELECT</strong> name , count(*) <s..."
885,network_1,"SELECT T2.name , count(*) FROM Friend AS T1 J...",What are the names of the high schoolers and h...,"[SELECT, T2.name, ,, count, (, *, ), FROM, Fri...","[select, t2, ., name, ,, count, (, *, ), from,...","[What, are, the, names, of, the, high, schoole...","""Write a query that answers the following ques...","\n\nSELECT firstname , lastname , COUNT(*)..."
886,network_1,SELECT T2.name FROM Friend AS T1 JOIN Highscho...,What is the name of the high schooler who has ...,"[SELECT, T2.name, FROM, Friend, AS, T1, JOIN, ...","[select, t2, ., name, from, friend, as, t1, jo...","[What, is, the, name, of, the, high, schooler,...","""Write a query that answers the following ques...",\n\nSELECT name FROM student WHERE count ...
887,network_1,SELECT T2.name FROM Friend AS T1 JOIN Highscho...,Return the name of the high school student wit...,"[SELECT, T2.name, FROM, Friend, AS, T1, JOIN, ...","[select, t2, ., name, from, friend, as, t1, jo...","[Return, the, name, of, the, high, school, stu...","""Write a query that answers the following ques...",\n\nSELECT name FROM student GROUP BY name O...
888,network_1,SELECT T2.name FROM Friend AS T1 JOIN Highscho...,Show the names of high schoolers who have at l...,"[SELECT, T2.name, FROM, Friend, AS, T1, JOIN, ...","[select, t2, ., name, from, friend, as, t1, jo...","[Show, the, names, of, high, schoolers, who, h...","""Write a query that answers the following ques...",<strong>SELECT</strong> name <strong>FROM</...
889,network_1,SELECT T2.name FROM Friend AS T1 JOIN Highscho...,What are the names of high schoolers who have ...,"[SELECT, T2.name, FROM, Friend, AS, T1, JOIN, ...","[select, t2, ., name, from, friend, as, t1, jo...","[What, are, the, names, of, high, schoolers, w...","""Write a query that answers the following ques...",\n\nSELECT name FROM student WHERE count(n...
890,network_1,SELECT T3.name FROM Friend AS T1 JOIN Highscho...,Show the names of all of the high schooler Kyl...,"[SELECT, T3.name, FROM, Friend, AS, T1, JOIN, ...","[select, t3, ., name, from, friend, as, t1, jo...","[Show, the, names, of, all, of, the, high, sch...","""Write a query that answers the following ques...",<strong>SELECT</strong> name <strong>FROM</...


(51, 7)
Input:  ['"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Which professionals have done at least two treatments? List the professional\'s id, role, and first name. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What are the id, role, and first name of the professionals who have performed two or more treatments? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the name of the breed with the most dogs? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Which breed do the most dogs have? Give me the breed name. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Which owner has p

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question,generated_query
932,dog_kennels,"SELECT T1.professional_id , T1.role_code , T...",Which professionals have done at least two tre...,"[SELECT, T1.professional_id, ,, T1.role_code, ...","[select, t1, ., professional_id, ,, t1, ., rol...","[Which, professionals, have, done, at, least, ...","""Write a query that answers the following ques...","\n\nSELECT prof_id , role , first_name FROM ..."
933,dog_kennels,"SELECT T1.professional_id , T1.role_code , T...","What are the id, role, and first name of the p...","[SELECT, T1.professional_id, ,, T1.role_code, ...","[select, t1, ., professional_id, ,, t1, ., rol...","[What, are, the, id, ,, role, ,, and, first, n...","""Write a query that answers the following ques...","\n\nSELECT t1.id , t1.role , t1.first_name F..."
934,dog_kennels,SELECT T1.breed_name FROM Breeds AS T1 JOIN Do...,What is the name of the breed with the most dogs?,"[SELECT, T1.breed_name, FROM, Breeds, AS, T1, ...","[select, t1, ., breed_name, from, breeds, as, ...","[What, is, the, name, of, the, breed, with, th...","""Write a query that answers the following ques...",\n\nSELECT breed_name FROM breeds ORDER BY do...
935,dog_kennels,SELECT T1.breed_name FROM Breeds AS T1 JOIN Do...,Which breed do the most dogs have? Give me the...,"[SELECT, T1.breed_name, FROM, Breeds, AS, T1, ...","[select, t1, ., breed_name, from, breeds, as, ...","[Which, breed, do, the, most, dogs, have, ?, G...","""Write a query that answers the following ques...",\n\nSELECT breed FROM dogs ORDER BY count DES...
936,dog_kennels,"SELECT T1.owner_id , T1.last_name FROM Owners...",Which owner has paid for the most treatments o...,"[SELECT, T1.owner_id, ,, T1.last_name, FROM, O...","[select, t1, ., owner_id, ,, t1, ., last_name,...","[Which, owner, has, paid, for, the, most, trea...","""Write a query that answers the following ques...","\n\nSELECT owner_id , last_name FROM owners..."
937,dog_kennels,"SELECT T1.owner_id , T1.last_name FROM Owners...",Tell me the owner id and last name of the owne...,"[SELECT, T1.owner_id, ,, T1.last_name, FROM, O...","[select, t1, ., owner_id, ,, t1, ., last_name,...","[Tell, me, the, owner, id, and, last, name, of...","""Write a query that answers the following ques...","\n\nSELECT owner_id , last_name FROM owners W..."
938,dog_kennels,SELECT T1.treatment_type_description FROM Trea...,What is the description of the treatment type ...,"[SELECT, T1.treatment_type_description, FROM, ...","[select, t1, ., treatment_type_description, fr...","[What, is, the, description, of, the, treatmen...","""Write a query that answers the following ques...",\n\nSELECT treatment_type_description FROM t...
939,dog_kennels,SELECT T1.treatment_type_description FROM Trea...,Give me the description of the treatment type ...,"[SELECT, T1.treatment_type_description, FROM, ...","[select, t1, ., treatment_type_description, fr...","[Give, me, the, description, of, the, treatmen...","""Write a query that answers the following ques...",\n\nSELECT treatment_type_description FROM t...
940,dog_kennels,"SELECT T1.owner_id , T1.zip_code FROM Owners ...",Which owner has paid the largest amount of mon...,"[SELECT, T1.owner_id, ,, T1.zip_code, FROM, Ow...","[select, t1, ., owner_id, ,, t1, ., zip_code, ...","[Which, owner, has, paid, the, largest, amount...","""Write a query that answers the following ques...","\n\nSELECT owner_id , zip_code FROM owners..."
941,dog_kennels,"SELECT T1.owner_id , T1.zip_code FROM Owners ...",Find the owner id and zip code of the owner wh...,"[SELECT, T1.owner_id, ,, T1.zip_code, FROM, Ow...","[select, t1, ., owner_id, ,, t1, ., zip_code, ...","[Find, the, owner, id, and, zip, code, of, the...","""Write a query that answers the following ques...","\n\nSELECT owner_id , zip_code FROM owner ..."


(51, 7)
Input:  ['"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> Find the number of professionals who have not treated any dogs. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> List the dog name, age and weight of the dogs who have been abandoned? 1 stands for yes, and 0 stands for no. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What are the dog name, age and weight of the dogs that were abandoned? Note that 1 stands for yes, and 0 stands for no in the tables. <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."\n\n<question> What is the average age of all the dogs? <query> ', '"Write a query that answers the following question. Return only the query and no additional explanation."

,db_id,query,question,query_toks,query_toks_no_value,question_toks,modified question,generated_query
983,dog_kennels,SELECT count(*) FROM Professionals WHERE profe...,Find the number of professionals who have not ...,"[SELECT, count, (, *, ), FROM, Professionals, ...","[select, count, (, *, ), from, professionals, ...","[Find, the, number, of, professionals, who, ha...","""Write a query that answers the following ques...",\n\nSELECT COUNT(*) FROM professionals WHERE ...
984,dog_kennels,"SELECT name , age , weight FROM Dogs WHERE a...","List the dog name, age and weight of the dogs ...","[SELECT, name, ,, age, ,, weight, FROM, Dogs, ...","[select, name, ,, age, ,, weight, from, dogs, ...","[List, the, dog, name, ,, age, and, weight, of...","""Write a query that answers the following ques...","<strong>SELECT dog_name, age, weight FROM dog..."
985,dog_kennels,"SELECT name , age , weight FROM Dogs WHERE a...","What are the dog name, age and weight of the d...","[SELECT, name, ,, age, ,, weight, FROM, Dogs, ...","[select, name, ,, age, ,, weight, from, dogs, ...","[What, are, the, dog, name, ,, age, and, weigh...","""Write a query that answers the following ques...","<strong>SELECT dog_name, age, weight FROM dog..."
986,dog_kennels,SELECT avg(age) FROM Dogs,What is the average age of all the dogs?,"[SELECT, avg, (, age, ), FROM, Dogs]","[select, avg, (, age, ), from, dogs]","[What, is, the, average, age, of, all, the, do...","""Write a query that answers the following ques...",\n\nSELECT AVG(age) FROM dog
987,dog_kennels,SELECT avg(age) FROM Dogs,Compute the average age of all the dogs.,"[SELECT, avg, (, age, ), FROM, Dogs]","[select, avg, (, age, ), from, dogs]","[Compute, the, average, age, of, all, the, dog...","""Write a query that answers the following ques...",<strong>SELECT AVG(age) FROM dog</strong>
988,dog_kennels,SELECT max(age) FROM Dogs,What is the age of the oldest dog?,"[SELECT, max, (, age, ), FROM, Dogs]","[select, max, (, age, ), from, dogs]","[What, is, the, age, of, the, oldest, dog, ?]","""Write a query that answers the following ques...",\n\nSELECT MAX(age) FROM dog
989,dog_kennels,SELECT max(age) FROM Dogs,Tell me the age of the oldest dog.,"[SELECT, max, (, age, ), FROM, Dogs]","[select, max, (, age, ), from, dogs]","[Tell, me, the, age, of, the, oldest, dog, .]","""Write a query that answers the following ques...",<strong>SELECT age FROM dog ORDER BY age DE...
990,dog_kennels,"SELECT charge_type , charge_amount FROM Charges",How much does each charge type costs? List bot...,"[SELECT, charge_type, ,, charge_amount, FROM, ...","[select, charge_type, ,, charge_amount, from, ...","[How, much, does, each, charge, type, costs, ?...","""Write a query that answers the following ques...","\n\nSELECT charge_type , SUM(amount) FROM cha..."
991,dog_kennels,"SELECT charge_type , charge_amount FROM Charges",List each charge type and its amount.,"[SELECT, charge_type, ,, charge_amount, FROM, ...","[select, charge_type, ,, charge_amount, from, ...","[List, each, charge, type, and, its, amount, .]","""Write a query that answers the following ques...","\n\nSELECT charge_type , SUM(amount) FROM cha..."
992,dog_kennels,SELECT max(charge_amount) FROM Charges,How much does the most expensive charge type c...,"[SELECT, max, (, charge_amount, ), FROM, Charges]","[select, max, (, charge_amount, ), from, charges]","[How, much, does, the, most, expensive, charge...","""Write a query that answers the following ques...",\n\nSELECT MAX(charge_type_cost) FROM charge_...


In [ ]:
final_df = pd.concat(output)
final_df.to_csv("/content/drive/MyDrive/NLP/gemma2b_finetuned_output.csv")

In [ ]:
#Megha
from unsloth import FastLanguageModel
from datasets import load_dataset

max_new_tokens = 2048
max_seq_length = 2048
dtype = None
load_in_4bit = True

spider_prompt = """"Write a query that answers the following question. Return only the query and no additional explanation."

<question> {} <query> {}"""

data_col = "question"

test_data = load_dataset("xlangai/spider", split = "validation")

queries = []

for i in range(max(10,len(test_data))):
    print(f"\n------------------------------------------------------------- Example {i} -----------------------------------------------------------\n")
    question = test_data[i][data_col]

    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    inputs = tokenizer(
    [
        spider_prompt.format(question,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    tokenizer.batch_decode(outputs)

    outputs = model.generate(**inputs, max_new_tokens = max_new_tokens, use_cache = True)
    modified_text = tokenizer.batch_decode(outputs)

    query = test_data[i]["query"]
    print(f"Actual query:\n{query}")
    print("="*100)
    queries.append(modified_text[0].split("<answer>")[-1])
    print("\nGenerated text:\n",modified_text[0].split("<answer>")[-1])


Streaming output truncated to the last 5000 lines.
 <bos>"Write a query that answers the following question. Return only the query and no additional explanation."

<question> What is the average, minimum, and maximum age for all French singers? <query> <strong>SELECT avg(age) ,  min(age) ,  max(age) FROM Singer WHERE nationality  =  "French"</strong><eos>

------------------------------------------------------------- Example 6 -----------------------------------------------------------

Actual query:
SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

Generated text:
 <bos>"Write a query that answers the following question. Return only the query and no additional explanation."

<question> Show the name and the release year of the song by the youngest singer. <query> <strong>SELECT T1.name ,  T1.release_year FROM singer AS T1 JOIN song AS T2 ON T1.singer_id  =  T2.singer_id ORDER BY T1.age LIMIT 1</strong><eos>

-------------------------------------------------------

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
len(queries)

460

In [ ]:
mod_queries = []
for q in range(len(queries)):
  print(f"Actual query:\n{test_data[q]['query']}")
  print(f"Generated text:\n{queries[q].split('<query>')[-1]}")
  mod_queries.append(queries[q].split('<query>')[-1])
  print("="*100)

Actual query:
SELECT count(*) FROM singer
Generated text:
 <strong>SELECT count(*) FROM Singer</strong><eos>
Actual query:
SELECT count(*) FROM singer
Generated text:
 <strong>SELECT count(*) FROM Singer</strong><eos>
Actual query:
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
Generated text:
 <strong>SELECT name ,  country ,  age FROM singer ORDER BY age DESC</strong><eos>
Actual query:
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
Generated text:
 <strong>SELECT name ,  country ,  age FROM Singer ORDER BY age DESC</strong><eos>
Actual query:
SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
Generated text:
 <strong>SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  "France"</strong><eos>
Actual query:
SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
Generated text:
 <strong>SELECT avg(age) ,  min(age) ,  max(age) FROM Singer WHERE nationality  =  "French"</strong><eos>
Actual que

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("xlangai/spider", split = "validation")
val_df = pd.DataFrame(dataset)
display(val_df.head())
val_df = val_df[:len(mod_queries)]

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]"
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]"
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m..."


In [ ]:
val_df["complete_generation"] = queries
val_df["generated_queries"] = mod_queries
val_df.head(10)

,db_id,query,question,query_toks,query_toks_no_value,question_toks,generated_queries,complete_generation
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]",<strong>SELECT count(*) FROM Singer</strong><...,"<bos>""Write a query that answers the following..."
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]",<strong>SELECT count(*) FROM Singer</strong><...,"<bos>""Write a query that answers the following..."
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin...","<strong>SELECT name , country , age FROM si...","<bos>""Write a query that answers the following..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ...","<strong>SELECT name , country , age FROM Si...","<bos>""Write a query that answers the following..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m...","<strong>SELECT avg(age) , min(age) , max(ag...","<bos>""Write a query that answers the following..."
5,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","[What, is, the, average, ,, minimum, ,, and, m...","<strong>SELECT avg(age) , min(age) , max(ag...","<bos>""Write a query that answers the following..."
6,concert_singer,"SELECT song_name , song_release_year FROM sin...",Show the name and the release year of the song...,"[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...","[Show, the, name, and, the, release, year, of,...","<strong>SELECT T1.name , T1.release_year FRO...","<bos>""Write a query that answers the following..."
7,concert_singer,"SELECT song_name , song_release_year FROM sin...",What are the names and release years for all t...,"[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...","[What, are, the, names, and, release, years, f...","<strong>SELECT T1.name , T1.release_year FRO...","<bos>""Write a query that answers the following..."
8,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,What are all distinct countries where singers ...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...","[What, are, all, distinct, countries, where, s...",<strong>SELECT DISTINCT Country FROM Singer W...,"<bos>""Write a query that answers the following..."
9,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,What are the different countries with singers...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...","[What, are, the, different, countries, with, s...",<strong>SELECT DISTINCT country FROM Singer W...,"<bos>""Write a query that answers the following..."


In [ ]:
val_df.to_csv("/content/drive/MyDrive/NLP/gemma7b_finetuned_output.csv",index=False)